In [41]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import geopy
from geopy.geocoders import Nominatim
from geopy.geocoders import Photon
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

## LOADING TRAINING AND TESTING DATASETS

Define some useful functions for fetching the datasets.

In [42]:
#function that replaces incorrect characters with the correct utf-8 characters
def correct_misinterpreted_characters(text):
    replacements = {
        "Ã¼": "ü",
        "Ã¶": "ö",
        "Ã¤": "ä",
        "Ã": "Ä"
    }
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

# Function to correct the misinterpreted characters in the CSV file content
def correct_csv_content(csv_content):
    corrected_content = []
    for row in csv_content:
        corrected_row = [correct_misinterpreted_characters(cell) for cell in row]
        corrected_content.append(corrected_row)
    return corrected_content


In [43]:
#returns the data of a specific year, corrects the encoding of the year 2022 (might be better solutions)
#removes missing data (completely random missing data, ones that fail to evaluate)
def import_data(year, unwanted_columns=[]):
    path = "data\\"+str(year)
    df_reisende = pd.read_csv(path+'\REISENDE.csv',sep=";",encoding = "utf-8")
    df_linie = pd.read_csv(path+'\LINIE.csv',sep=";",encoding = "utf-8")

    #correcting characters on year 2022
    if year == 2022:
        with open(path+'\HALTESTELLEN.csv', 'r', encoding='utf-8') as f:
            csv_content = list(csv.reader(f, delimiter=';'))
            corrected_content = correct_csv_content(csv_content[1:])
            df_haltestellen = pd.DataFrame(corrected_content[1:], columns=csv_content[0])
        df_haltestellen['Haltestellen_Id'] = df_haltestellen['Haltestellen_Id'].astype(int)
    else:
        df_haltestellen = pd.read_csv(path+'\HALTESTELLEN.csv',sep=";",encoding = "utf-8")

    # Merge into one table
    merged_df = df_reisende.merge(df_haltestellen, left_on='Haltestellen_Id', right_on='Haltestellen_Id')
    merged_df = merged_df.merge(df_haltestellen, left_on='Nach_Hst_Id', right_on='Haltestellen_Id', suffixes=('_from','_to'))
    merged_df = merged_df.merge(df_linie[["Linien_Id", "VSYS"]], left_on='Linien_Id', right_on='Linien_Id') #to obtain the vehicle type
    
    #finally drop unwanted columns
    merged_df.dropna(inplace=True)
    if len(unwanted_columns) > 0:
        merged_df.drop(unwanted_columns, axis=1, inplace=True)
    
    return merged_df.dropna(), df_haltestellen, df_linie

#### Let's see the percentage of missing data

In [44]:
for i in range(2015, 2024):
    dftmp=pd.read_csv(f'data/{i}/REISENDE.csv',sep=";",encoding = "utf-8")
    original_size=len(dftmp)
    
    dfnan=dftmp[dftmp.isna().any(axis=1)]
    nan_size=len(dfnan)
    
    print(f'Year {i} with percentage of missing data:{nan_size/original_size*100:.2f}%')

Year 2015 with percentage of missing data:5.42%
Year 2016 with percentage of missing data:5.46%
Year 2017 with percentage of missing data:5.35%
Year 2018 with percentage of missing data:5.22%
Year 2019 with percentage of missing data:5.47%
Year 2020 with percentage of missing data:5.82%
Year 2021 with percentage of missing data:5.02%
Year 2022 with percentage of missing data:5.35%
Year 2023 with percentage of missing data:5.36%


Define unwanted columns for train and test datasets.

In [45]:
unwanted_columns=[
 'Anzahl_Messungen',
 'Aussteiger',
 'Distanz',
 'Einsteiger',
 'Haltestellen_Id_from',
 'Haltestellen_Id_to',
 'Haltestellennummer_from',
 'Haltestellennummer_to',
 'Haltestellenlangname_from',
 'Haltestellenlangname_to',                
 'ID_Abschnitt',
 'Linien_Id',
 'Nach_Hst_Id',
 'Nachtnetz',
 'Plan_Fahrt_Id',
 'Richtung',
 'Sequenz',
 'Tagtyp_Id',
 'Tage_DTV']

Now fetsch the training and testing datasets.

In [46]:
# setting dataset of year 2022 as training set and 2023 as testing set
df_train, stops_train, lines_train=import_data(2022, unwanted_columns)
df_test, stops_test, lines_test=import_data(2023, unwanted_columns)

In [47]:
# find common stops and stops that are added in 2023 using set intersection
set_stop_train=set(stops_train["Haltestellenkurzname"])
set_stop_test=set(stops_test["Haltestellenkurzname"])
                                                                    #to include stops that are added in 2023
common_stops=set_stop_train.intersection(set_stop_test).union(set_stop_test.difference(set_stop_train))

#find common linenames using set intersection
set_lines_train=set(lines_train["Linienname"])
set_lines_test=set(lines_train["Linienname"])
common_lines=set_lines_train.intersection(set_lines_test)


In [48]:
#filter train dataset on...
    #stops
df_filtered_train = df_train[df_train['Haltestellenkurzname_from'].isin(common_stops)]
df_filtered_train = df_train[df_train['Haltestellenkurzname_to'].isin(common_stops)]
    #line names
df_filtered_train = df_train[df_train['Linienname'].isin(common_lines)]

#filter test dataset on...
    #stops
df_filtered_test = df_test[df_test['Haltestellenkurzname_from'].isin(common_stops)]
df_filtered_test = df_test[df_test['Haltestellenkurzname_to'].isin(common_stops)]
    #line names
df_filtered_test = df_test[df_test['Linienname'].isin(common_lines)]


## CONVERTING TIME TO BINS OF 30 MINUTES

In [49]:
def convert_time_to_bins(time):
    # Convert time to hours
    hour, minute, _ = map(int, time.split(':'))
    time_in_hours = hour + minute / 60
    
    # Calculate time modulo 24
    time_modulo_24 = time_in_hours % 24
    
    # Calculate bin index
    bin_index = int(time_modulo_24 * 2) / 2  # Round to nearest half-hour
    
    return bin_index
    
df_filtered_train["FZ_AB"]=df_filtered_train["FZ_AB"].apply(convert_time_to_bins)
df_filtered_test["FZ_AB"]=df_filtered_test["FZ_AB"].apply(convert_time_to_bins)

C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\4095238205.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_test["FZ_AB"]=df_filtered_test["FZ_AB"].apply(convert_time_to_bins)


## CONVERT TAG DAYS TO BOOLEAN VALUES, TO KNOW THE DAY TYPES

In [50]:
#convert the tag days to boolean values, to know which is the day type, this operation is similar to the result of a one-hot encoding

def division(n):
    return False if n == 0 else True

df_filtered_train['Tage_DWV']=df_filtered_train['Tage_DWV'].apply(division)
df_filtered_train['Tage_SA']=df_filtered_train['Tage_SA'].apply(division)
df_filtered_train['Tage_SO']=df_filtered_train['Tage_SO'].apply(division)
df_filtered_train['Tage_SA_N']=df_filtered_train['Tage_SA_N'].apply(division)
df_filtered_train['Tage_SO_N']=df_filtered_train['Tage_SO_N'].apply(division)
df_filtered_train

,Linienname,FZ_AB,Besetzung,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,Haltestellenkurzname_from,Haltestellenkurzname_to,VSYS
0,78,21.0,3.22000,True,False,False,False,False,GRUN,BALT,B
1,78,21.5,2.83636,True,False,False,False,False,GRUN,BALT,B
2,78,21.5,3.10417,True,False,False,False,False,GRUN,BALT,B
3,78,22.0,3.06897,True,False,False,False,False,GRUN,BALT,B
4,78,22.0,2.57143,True,False,False,False,False,GRUN,BALT,B
...,...,...,...,...,...,...,...,...,...,...,...
974678,25,22.0,4.86957,False,True,False,False,False,WDOL,BDOL,SB
974679,25,22.5,4.60000,False,True,False,False,False,WDOL,BDOL,SB
974680,25,22.5,4.08696,False,True,False,False,False,WDOL,BDOL,SB
974681,25,23.0,1.81818,False,True,False,False,False,WDOL,BDOL,SB


In [51]:
#same of test set
df_filtered_test['Tage_DWV']=df_filtered_test['Tage_DWV'].apply(division)
df_filtered_test['Tage_SA']=df_filtered_test['Tage_SA'].apply(division)
df_filtered_test['Tage_SO']=df_filtered_test['Tage_SO'].apply(division)
df_filtered_test['Tage_SA_N']=df_filtered_test['Tage_SA_N'].apply(division)
df_filtered_test['Tage_SO_N']=df_filtered_test['Tage_SO_N'].apply(division)
df_filtered_test

C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\3078013300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_test['Tage_DWV']=df_filtered_test['Tage_DWV'].apply(division)
C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\3078013300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_test['Tage_SA']=df_filtered_test['Tage_SA'].apply(division)
C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\3078013300.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,Linienname,FZ_AB,Besetzung,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,Haltestellenkurzname_from,Haltestellenkurzname_to,VSYS
0,15,9.5,0.44444,False,False,True,False,False,MIBU,HIRS,T
1,15,10.0,1.60000,False,False,True,False,False,MIBU,HIRS,T
2,15,5.0,0.50000,True,False,False,False,False,MIBU,HIRS,T
3,15,5.0,0.10000,True,False,False,False,False,MIBU,HIRS,T
4,15,5.5,0.50000,True,False,False,False,False,MIBU,HIRS,T
...,...,...,...,...,...,...,...,...,...,...,...
1149500,315,13.0,5.97826,True,False,False,False,False,KIND,EIHH,BL
1149501,315,12.0,11.05556,True,False,False,False,False,EIHH,KIND,BL
1149502,315,15.5,6.93182,True,False,False,False,False,EIHH,KIND,BL
1149503,315,16.5,4.26667,True,False,False,False,False,EIHH,KIND,BL


## DUMMIFY LINENAME TO BOOLEAN VALUES

In [52]:
#convert LInename to numerical values
df_filtered_train = pd.get_dummies(df_filtered_train, columns=['Linienname'], prefix='Line')
df_filtered_train

,FZ_AB,Besetzung,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,Haltestellenkurzname_from,Haltestellenkurzname_to,VSYS,...,Line_743,Line_744,Line_745,Line_751,Line_753,Line_910,Line_912,Line_916,Line_918,Line_919
0,21.0,3.22000,True,False,False,False,False,GRUN,BALT,B,...,False,False,False,False,False,False,False,False,False,False
1,21.5,2.83636,True,False,False,False,False,GRUN,BALT,B,...,False,False,False,False,False,False,False,False,False,False
2,21.5,3.10417,True,False,False,False,False,GRUN,BALT,B,...,False,False,False,False,False,False,False,False,False,False
3,22.0,3.06897,True,False,False,False,False,GRUN,BALT,B,...,False,False,False,False,False,False,False,False,False,False
4,22.0,2.57143,True,False,False,False,False,GRUN,BALT,B,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974678,22.0,4.86957,False,True,False,False,False,WDOL,BDOL,SB,...,False,False,False,False,False,False,False,False,False,False
974679,22.5,4.60000,False,True,False,False,False,WDOL,BDOL,SB,...,False,False,False,False,False,False,False,False,False,False
974680,22.5,4.08696,False,True,False,False,False,WDOL,BDOL,SB,...,False,False,False,False,False,False,False,False,False,False
974681,23.0,1.81818,False,True,False,False,False,WDOL,BDOL,SB,...,False,False,False,False,False,False,False,False,False,False


In [53]:
#now the same for test set
df_filtered_test = pd.get_dummies(df_filtered_test, columns=['Linienname'], prefix='Line')
df_filtered_test

,FZ_AB,Besetzung,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,Haltestellenkurzname_from,Haltestellenkurzname_to,VSYS,...,Line_743,Line_744,Line_745,Line_751,Line_753,Line_910,Line_912,Line_916,Line_918,Line_919
0,9.5,0.44444,False,False,True,False,False,MIBU,HIRS,T,...,False,False,False,False,False,False,False,False,False,False
1,10.0,1.60000,False,False,True,False,False,MIBU,HIRS,T,...,False,False,False,False,False,False,False,False,False,False
2,5.0,0.50000,True,False,False,False,False,MIBU,HIRS,T,...,False,False,False,False,False,False,False,False,False,False
3,5.0,0.10000,True,False,False,False,False,MIBU,HIRS,T,...,False,False,False,False,False,False,False,False,False,False
4,5.5,0.50000,True,False,False,False,False,MIBU,HIRS,T,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149500,13.0,5.97826,True,False,False,False,False,KIND,EIHH,BL,...,False,False,False,False,False,False,False,False,False,False
1149501,12.0,11.05556,True,False,False,False,False,EIHH,KIND,BL,...,False,False,False,False,False,False,False,False,False,False
1149502,15.5,6.93182,True,False,False,False,False,EIHH,KIND,BL,...,False,False,False,False,False,False,False,False,False,False
1149503,16.5,4.26667,True,False,False,False,False,EIHH,KIND,BL,...,False,False,False,False,False,False,False,False,False,False


## ADD GEOLOGICAL COORDINATES TO HALTESTELLEN FROM AND HALTESTELLEN TO

#### RETRIEVING GPS COORDINATES FOR ALL STOPS

First we find all possible stops that are in both train and test datasets.

We use haltestellen-kurzname to reference to keep a better consistency across different years,
and we use haltestellen-langname specifically to look for the GPS coordinates. 

In [54]:
set_stops_train = {(a, b) for a, b in zip(stops_train['Haltestellenkurzname'], stops_train['Haltestellenlangname'])}
set_stops_test = {(a, b) for a, b in zip(stops_test['Haltestellenkurzname'], stops_test['Haltestellenlangname'])}


In [55]:
stops_train_test=set_stops_train.union(set_stops_test)
df_stops_train_test=pd.DataFrame(stops_train_test, columns=["Haltestellenkurzname", "Haltestellenlangname"])

In [56]:
#import stops with coordinates
df_stops_with_coordinates=pd.read_csv("data/stops.csv")[["Haltestellenlangname","GPS_Latitude","GPS_Longitude"]]

In [57]:
df_all_stops=pd.merge(df_stops_train_test, df_stops_with_coordinates, on='Haltestellenlangname', how='left')

In [58]:
df_all_stops[df_all_stops["GPS_Latitude"].isna()]

,Haltestellenkurzname,Haltestellenlangname,GPS_Latitude,GPS_Longitude
8,WALBIR,"Waldegg, Birmensdorferstrasse",NaN,NaN
11,ZUMI99,Zumikon,NaN,NaN
12,KLAE07,"Kloten, Lägernstrasse",NaN,NaN
20,VBIR,"Villmergen, Birkenweg",NaN,NaN
23,BIRWSL,"Birmensdorf ZH, Sternen/WSL",NaN,NaN
...,...,...,...,...
795,WIDDOR,"Widen, Dorf",NaN,NaN
799,BERMAT,"Berikon, Mattenhof",NaN,NaN
800,BALD01,"Adliswil, Baldernstrasse",NaN,NaN
802,SVOR,"Sarmenstorf, Vorstadt",NaN,NaN


In [59]:
#now find the latitude and longitude of those without, from stops.csv

In [60]:
# initialize Nominatim geocoder
geolocator = Photon(user_agent="geopy_project")

# function for rows that have null GPS coordinates
def geocoord_of(row):
    coordinates = geolocator.geocode(row["Haltestellenlangname"]+", Zurich")
    if coordinates is not None:
        print(row["Haltestellenlangname"]+": ",coordinates.latitude, coordinates.longitude)
        return coordinates.latitude, coordinates.longitude
    else:
        return None, None  # Return None values if geocoding fails

# apply the function only to rows where column GPS latitude (or longitude, whicever of the two) is NaN
df_all_stops[['GPS_Latitude', 'GPS_Longitude']] = df_all_stops.apply(lambda row: geocoord_of(row) if pd.isnull(row['GPS_Latitude']) else (row['GPS_Latitude'], row['GPS_Longitude']), axis=1, result_type='expand')


Waldegg, Birmensdorferstrasse:  47.3688877 8.4644298
Zumikon:  47.3324061 8.6238228
Kloten, Lägernstrasse:  47.4468522 8.5768629
Villmergen, Birkenweg:  47.3513653 8.2505702
Birmensdorf ZH, Sternen/WSL:  47.360094 8.456101
Oetwil a.d.L., Schweizäcker:  47.4245207 8.402553
Dietikon, Wasserburg:  47.3988505 8.4298643
Oberwil-Lieli, Dreispitz:  47.3372025 8.3866316
Wohlen AG, Freiämterhof:  47.349128 8.2639433
Villmergen, Feldblumenweg:  47.3496055 8.2470707
Waltikon:  47.3354762 8.6188248
Zürich, Zielweg:  47.3588812 8.4973394
Bremgarten AG, Bahnhof:  47.351740649999996 8.346632609937817
Zollikerberg, Sennhof:  47.3462027 8.6171916
Zürich, Stodolastrasse:  47.3572507 8.590993
Scheuren:  47.3225026 8.6606431
Wohlen AG, Coop:  47.3501784 8.2709925
Wohlen AG, Haldenschulen:  47.3506379 8.2801625
Zürich Flughafen, Werft:  47.4453771 8.5703427
Hilfikon, Sandbüel:  47.3318031 8.8643112
Kloten, Neubrunnenstrasse:  47.4452507 8.5761344
Zollikerberg, Station/Q treff:  47.3465737 8.6036099
Zufikon

In [21]:
#finding stops which the process of fetching geological coordinates failed 

In [61]:
df_all_stops[df_all_stops["GPS_Latitude"].isna() ]

,Haltestellenkurzname,Haltestellenlangname,GPS_Latitude,GPS_Longitude
52,ENGEPAU,"Zürich, Bhf Enge Pausenplatz",NaN,NaN
169,BWST,"Zch, Bhf.Wollishofen/Staubstr.",NaN,NaN


In [62]:
#we discovered that the stop "Zürich, Bhf Enge Pausenplatz" was never referenced in the Reisende.csv table in both 2022 and 2023
#so we remove it
df_all_stops.drop(52, inplace=True)

In [63]:

df_all_stops[df_all_stops["GPS_Latitude"].isna() ]

,Haltestellenkurzname,Haltestellenlangname,GPS_Latitude,GPS_Longitude
169,BWST,"Zch, Bhf.Wollishofen/Staubstr.",NaN,NaN


In [64]:
#now manually insert the coordinates of "Zch, Bhf.Wollishofen/Staubstr", with coordinates 47.34703438638321, 8.532916780705717
df_all_stops.at[169, 'GPS_Latitude'] = 47.34703438638321
df_all_stops.at[169, 'GPS_Longitude'] = 8.532916780705717

In [65]:
#verify that there is no non values left in geological coordinates
df_all_stops[df_all_stops["GPS_Latitude"].isna() ]

,Haltestellenkurzname,Haltestellenlangname,GPS_Latitude,GPS_Longitude


In [66]:
#now we can drop the columns Haltestellenlangname from the df_all_stops table
df_all_stops.drop(columns=["Haltestellenlangname"],inplace=True)


#### ADDING GPS COORDINATES TO THE DATASETS

In [67]:
#now we can merge the reisende table (df_filtered_train and df_filtered_test) with df_all_Stops, to get all the coordinates.
df_filtered_train_ready = pd.merge(df_filtered_train, df_all_stops, left_on='Haltestellenkurzname_from', right_on='Haltestellenkurzname')
df_filtered_train_ready.rename(columns={'GPS_Latitude': 'GPS_Latitude_From', 'GPS_Longitude': 'GPS_Longitude_From'}, inplace=True)
df_filtered_train_ready = pd.merge(df_filtered_train_ready, df_all_stops, left_on='Haltestellenkurzname_to', right_on='Haltestellenkurzname')
df_filtered_train_ready.rename(columns={'GPS_Latitude': 'GPS_Latitude_To', 'GPS_Longitude': 'GPS_Longitude_To'}, inplace=True)

In [68]:
df_filtered_train_ready.drop(columns=['Haltestellenkurzname_from'], inplace=True)
df_filtered_train_ready.drop(columns=['Haltestellenkurzname_to'], inplace=True)
df_filtered_train_ready.drop(columns=['Haltestellenkurzname_x'], inplace=True)
df_filtered_train_ready.drop(columns=['Haltestellenkurzname_y'], inplace=True)

In [69]:
df_filtered_train_ready

,FZ_AB,Besetzung,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,VSYS,Line_2,Line_3,...,Line_753,Line_910,Line_912,Line_916,Line_918,Line_919,GPS_Latitude_From,GPS_Longitude_From,GPS_Latitude_To,GPS_Longitude_To
0,21.0,3.22000,True,False,False,False,False,B,False,False,...,False,False,False,False,False,False,47.394903,8.489476,47.390838,8.488814
1,21.5,2.83636,True,False,False,False,False,B,False,False,...,False,False,False,False,False,False,47.394903,8.489476,47.390838,8.488814
2,21.5,3.10417,True,False,False,False,False,B,False,False,...,False,False,False,False,False,False,47.394903,8.489476,47.390838,8.488814
3,22.0,3.06897,True,False,False,False,False,B,False,False,...,False,False,False,False,False,False,47.394903,8.489476,47.390838,8.488814
4,22.0,2.57143,True,False,False,False,False,B,False,False,...,False,False,False,False,False,False,47.394903,8.489476,47.390838,8.488814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972919,22.0,4.86957,False,True,False,False,False,SB,False,False,...,False,False,False,False,False,False,47.373257,8.568617,47.373789,8.576132
972920,22.5,4.60000,False,True,False,False,False,SB,False,False,...,False,False,False,False,False,False,47.373257,8.568617,47.373789,8.576132
972921,22.5,4.08696,False,True,False,False,False,SB,False,False,...,False,False,False,False,False,False,47.373257,8.568617,47.373789,8.576132
972922,23.0,1.81818,False,True,False,False,False,SB,False,False,...,False,False,False,False,False,False,47.373257,8.568617,47.373789,8.576132


In [70]:
# #now do the same with test data

# #now we can merge the reisende table (df_filtered_train and df_filtered_test) with df_all_Stops, to get all the coordinates.
df_filtered_test_ready = pd.merge(df_filtered_test, df_all_stops, left_on='Haltestellenkurzname_from', right_on='Haltestellenkurzname')
df_filtered_test_ready.rename(columns={'GPS_Latitude': 'GPS_Latitude_From', 'GPS_Longitude': 'GPS_Longitude_From'}, inplace=True)
df_filtered_test_ready = pd.merge(df_filtered_test_ready, df_all_stops, left_on='Haltestellenkurzname_to', right_on='Haltestellenkurzname')
df_filtered_test_ready.rename(columns={'GPS_Latitude': 'GPS_Latitude_To', 'GPS_Longitude': 'GPS_Longitude_To'}, inplace=True)

df_filtered_test_ready.drop(columns=['Haltestellenkurzname_from'], inplace=True)
df_filtered_test_ready.drop(columns=['Haltestellenkurzname_to'], inplace=True)
df_filtered_test_ready.drop(columns=['Haltestellenkurzname_x'], inplace=True)
df_filtered_test_ready.drop(columns=['Haltestellenkurzname_y'], inplace=True)

df_filtered_test_ready

,FZ_AB,Besetzung,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,VSYS,Line_2,Line_3,...,Line_753,Line_910,Line_912,Line_916,Line_918,Line_919,GPS_Latitude_From,GPS_Longitude_From,GPS_Latitude_To,GPS_Longitude_To
0,9.5,0.44444,False,False,True,False,False,T,False,False,...,False,False,False,False,False,False,47.397778,8.541750,47.400252,8.543470
1,10.0,1.60000,False,False,True,False,False,T,False,False,...,False,False,False,False,False,False,47.397778,8.541750,47.400252,8.543470
2,5.0,0.50000,True,False,False,False,False,T,False,False,...,False,False,False,False,False,False,47.397778,8.541750,47.400252,8.543470
3,5.0,0.10000,True,False,False,False,False,T,False,False,...,False,False,False,False,False,False,47.397778,8.541750,47.400252,8.543470
4,5.5,0.50000,True,False,False,False,False,T,False,False,...,False,False,False,False,False,False,47.397778,8.541750,47.400252,8.543470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140354,15.5,4.06250,False,True,False,False,False,BP,False,False,...,False,False,False,False,True,False,47.311703,8.588525,47.313352,8.588639
1140355,16.0,4.50000,False,True,False,False,False,BP,False,False,...,False,False,False,False,True,False,47.311703,8.588525,47.313352,8.588639
1140356,16.5,3.41176,False,True,False,False,False,BP,False,False,...,False,False,False,False,True,False,47.311703,8.588525,47.313352,8.588639
1140357,17.0,2.88235,False,True,False,False,False,BP,False,False,...,False,False,False,False,True,False,47.311703,8.588525,47.313352,8.588639


In [71]:
df_filtered_train_ready.to_csv("2022_training.csv", sep=';', encoding='utf-8')
df_filtered_test_ready.to_csv("2023_testing.csv", sep=';', encoding='utf-8')

In [72]:
df_filtered_train_ready[['FZ_AB','Besetzung','Tage_DWV','VSYS','GPS_Latitude_From','GPS_Longitude_From','Line_2','Line_3','Line_753','Line_910','Line_912','Line_916','Line_918','Line_919']]

,FZ_AB,Besetzung,Tage_DWV,VSYS,GPS_Latitude_From,GPS_Longitude_From,Line_2,Line_3,Line_753,Line_910,Line_912,Line_916,Line_918,Line_919
0,21.0,3.22000,True,B,47.394903,8.489476,False,False,False,False,False,False,False,False
1,21.5,2.83636,True,B,47.394903,8.489476,False,False,False,False,False,False,False,False
2,21.5,3.10417,True,B,47.394903,8.489476,False,False,False,False,False,False,False,False
3,22.0,3.06897,True,B,47.394903,8.489476,False,False,False,False,False,False,False,False
4,22.0,2.57143,True,B,47.394903,8.489476,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972919,22.0,4.86957,False,SB,47.373257,8.568617,False,False,False,False,False,False,False,False
972920,22.5,4.60000,False,SB,47.373257,8.568617,False,False,False,False,False,False,False,False
972921,22.5,4.08696,False,SB,47.373257,8.568617,False,False,False,False,False,False,False,False
972922,23.0,1.81818,False,SB,47.373257,8.568617,False,False,False,False,False,False,False,False


In [73]:
print(df_filtered_train_ready.columns)

Index(['FZ_AB', 'Besetzung', 'Tage_DWV', 'Tage_SA', 'Tage_SO', 'Tage_SA_N',
       'Tage_SO_N', 'VSYS', 'Line_2', 'Line_3',
       ...
       'Line_753', 'Line_910', 'Line_912', 'Line_916', 'Line_918', 'Line_919',
       'GPS_Latitude_From', 'GPS_Longitude_From', 'GPS_Latitude_To',
       'GPS_Longitude_To'],
      dtype='object', length=119)


## NORMALIZING AND INITIAL TESTING WITH LINEAR REGRESSION

In [74]:
trainset=df_filtered_train_ready
testset=df_filtered_test_ready

# Too large (particularly for svr), keep only tram
trainset = trainset[trainset['VSYS'] == 'T']
testset = testset[testset['VSYS'] ==  'T']

In [75]:
trainset.dropna(subset=trainset.columns.values, inplace=True)
testset.dropna(subset=testset.columns.values, inplace=True)

C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\2697619568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainset.dropna(subset=trainset.columns.values, inplace=True)
C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\2697619568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset.dropna(subset=testset.columns.values, inplace=True)


In [76]:
trainset.drop(columns=['Line_303','Line_308','Line_311','Line_331'], axis=1, inplace=True)
testset.drop(columns=['Line_332'], axis=1, inplace=True)

C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\625823002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainset.drop(columns=['Line_303','Line_308','Line_311','Line_331'], axis=1, inplace=True)
C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\625823002.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset.drop(columns=['Line_332'], axis=1, inplace=True)


In [77]:
scaler = StandardScaler()
columns_to_normalize = ['FZ_AB', 'GPS_Latitude_From', 'GPS_Latitude_To', 'GPS_Longitude_From', 'GPS_Longitude_To', 'Besetzung' ]
scaler.fit(trainset[columns_to_normalize])

# Transform both training and test sets
trainset[columns_to_normalize] = scaler.transform(trainset[columns_to_normalize])
testset[columns_to_normalize] = scaler.transform(testset[columns_to_normalize])

C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\3741256183.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainset[columns_to_normalize] = scaler.transform(trainset[columns_to_normalize])
C:\Users\TUF\AppData\Local\Temp\ipykernel_20184\3741256183.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset[columns_to_normalize] = scaler.transform(testset[columns_to_normalize])


In [78]:
X_train = trainset.drop(columns=['VSYS', 'Besetzung'])
y_train = trainset['Besetzung']
X_test = testset.drop(columns=['VSYS', 'Besetzung'])
y_test = testset['Besetzung']
X_train.head()

,FZ_AB,Tage_DWV,Tage_SA,Tage_SO,Tage_SA_N,Tage_SO_N,Line_2,Line_3,Line_4,Line_5,...,Line_753,Line_910,Line_912,Line_916,Line_918,Line_919,GPS_Latitude_From,GPS_Longitude_From,GPS_Latitude_To,GPS_Longitude_To
7288,0.825894,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0.720755,-1.979474,0.790881,-2.115646
7289,0.825894,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0.720755,-1.979474,0.790881,-2.115646
7290,0.915639,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0.720755,-1.979474,0.790881,-2.115646
7291,0.915639,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0.720755,-1.979474,0.790881,-2.115646
7292,0.646403,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,0.720755,-1.979474,0.790881,-2.115646


In [79]:
# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate errors
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"MAPE: {mape}%")

MAE: 0.8134882894111866
MSE: 1.1619050458174205
MAPE: 242.25472578631044%
